<a href="https://colab.research.google.com/github/BassamTar99/StockPrediction/blob/main/Samir_NewsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install newsapi-python vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import datetime
from newsapi import NewsApiClient
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Setup
api_key = 'e85a418d537146af9e862d574bd651d9'
newsapi = NewsApiClient(api_key=api_key)
analyzer = SentimentIntensityAnalyzer()


In [3]:
def get_stock_sentiment(ticker, days=7):
    # Date range
    today = datetime.date.today()
    start_date = today - datetime.timedelta(days=days)

    # Fetch news
    articles = newsapi.get_everything(
        q=ticker,
        from_param=start_date.isoformat(),
        to=today.isoformat(),
        language='en',
        sort_by='relevancy',
        page_size=100
    )

    # Process headlines
    data = []
    for article in articles['articles']:
        headline = article['title']
        date = article['publishedAt'][:10]
        score = analyzer.polarity_scores(headline)['compound']
        data.append({'date': date, 'headline': headline, 'sentiment': score})

    if not data:
        print("No news found.")
        return pd.DataFrame()

    df = pd.DataFrame(data)

    # Aggregate
    daily = df.groupby('date')['sentiment'].mean().reset_index()
    daily.columns = ['date', 'avg_sentiment']

    def sentiment_label(score):
        if score > 0.05:
            return 'positive'
        elif score < -0.05:
            return 'negative'
        else:
            return 'neutral'

    daily['tone'] = daily['avg_sentiment'].apply(sentiment_label)
    return daily


In [4]:
get_stock_sentiment("TSLA", days=3)


,date,avg_sentiment,tone
0,2025-04-15,-0.068719,negative
1,2025-04-16,0.171793,positive
2,2025-04-17,0.267933,positive
